- O que é o DES? (overview)
- Comparar com a cifra de Feistel
- Falar sobre confusão e difusão
- Explicar como ele funciona (black box)

In [544]:
from typing import List

Tabelas de permutação, expansão, etc utilizadas

In [545]:
initial_perm = [
    [58, 50, 42, 34, 26, 18, 10, 2],
    [60, 52, 44, 36, 28, 20, 12, 4],
    [62, 54, 46, 38, 30, 22, 14, 6],
    [64, 56, 48, 40, 32, 24, 16, 8],
    [57, 49, 41, 33, 25, 17, 9, 1],
    [59, 51, 43, 35, 27, 19, 11, 3],
    [61, 53, 45, 37, 29, 21, 13, 5],
    [63, 55, 47, 39, 31, 23, 15, 7],
]

final_perm = [
    [40, 8, 48, 16, 56, 24, 64, 32],
    [39, 7, 47, 15, 55, 23, 63, 31],
    [38, 6, 46, 14, 54, 22, 62, 30],
    [37, 5, 45, 13, 53, 21, 61, 29],
    [36, 4, 44, 12, 52, 20, 60, 28],
    [35, 3, 43, 11, 51, 19, 59, 27],
    [34, 2, 42, 10, 50, 18, 58, 26],
    [33, 1, 41, 9, 49, 17, 57, 25],
]

keyp = [
    [57, 49, 41, 33, 25, 17, 9],
    [1, 58, 50, 42, 34, 26, 18],
    [10, 2, 59, 51, 43, 35, 27],
    [19, 11, 3, 60, 52, 44, 36],
    [63, 55, 47, 39, 31, 23, 15],
    [7, 62, 54, 46, 38, 30, 22],
    [14, 6, 61, 53, 45, 37, 29],
    [21, 13, 5, 28, 20, 12, 4],
]

key_comp = [
    [14, 17, 11, 24, 1, 5],
    [3, 28, 15, 6, 21, 10],
    [23, 19, 12, 4, 26, 8],
    [16, 7, 27, 20, 13, 2],
    [41, 52, 31, 37, 47, 55],
    [30, 40, 51, 45, 33, 48],
    [44, 49, 39, 56, 34, 53],
    [46, 42, 50, 36, 29, 32],
]

dbox = [
    [32, 1, 2, 3, 4, 5, 4, 5],
    [6, 7, 8, 9, 8, 9, 10, 11],
    [12, 13, 12, 13, 14, 15, 16, 17],
    [16, 17, 18, 19, 20, 21, 20, 21],
    [22, 23, 24, 25, 24, 25, 26, 27],
    [28, 29, 28, 29, 30, 31, 32, 1],
]

pbox = [
    [16, 7, 20, 21],
    [29, 12, 28, 17],
    [1, 15, 23, 26],
    [5, 18, 31, 10],
    [2, 8, 24, 14],
    [32, 27, 3, 9],
    [19, 13, 30, 6],
    [22, 11, 4, 25],
]

shift_table = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

sbox = [
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],
    ],
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],
    ],
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],
    ],
    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],
    ],
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],
    ],
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],
    ],
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],
    ],
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],
    ],
]

Funções auxiliares para trabalhar com valores binários
Criei a classe bin porque o python ignora zeros a esquerda e o algoritmo precisa manter o tamanho dos
dados binários consistentes

In [546]:
class Bin:

    def __init__(self, value: int, size: int):
        self.value = value
        self.size = size

    def __repr__(self):
        return f"Bin(value={self.to_hex()}, size={self.size})"

    def __str__(self):
        return self.to_hex()

    def __eq__(self, other):
        if isinstance(other, Bin):
            return self.value == other.value and self.size == other.size
        return False

    def to_bin(self):
        return f"0b{self.value:0{self.size}b}"

    def to_hex(self):
        hex_digits = (self.size + 3) // 4
        return f"0x{self.value:0{hex_digits}X}"


def extract_bits(bits: Bin, table: List[List[int]]) -> Bin:
    extracted_bits = []
    for row in table:
        for bit_pos in row:
            bit = (bits.value >> (bits.size - bit_pos)) & 1
            extracted_bits.append(bit)

    result = 0
    for bit in extracted_bits:
        result = (result << 1) | bit

    return Bin(result, len(extracted_bits))


def halve(bits: Bin):
    half = (bits.size + 1) // 2
    left = (bits.value >> half) & ((1 << half) - 1)
    right = bits.value & ((1 << half) - 1)

    return Bin(left, half), Bin(right, half)


def split_bits(bits: Bin, chunk_size: int):
    chunks: List[Bin] = []
    for i in range(0, bits.size, chunk_size):
        chunk_value = (bits.value >> (bits.size - i - chunk_size)) & (
            (1 << chunk_size) - 1
        )
        chunks.append(Bin(chunk_value, chunk_size))
    return chunks


def fuse_bits(*bins: Bin):
    total_value = 0
    total_size = 0
    for b in bins:
        total_value = (total_value << b.size) | b.value
        total_size += b.size
    return Bin(total_value, total_size)


def swap_halves(bits: Bin):
    left, right = halve(bits)
    return fuse_bits(right, left)


def circular_left_shift(bits: Bin, shift_value: int):
    shift_value %= bits.size
    discarded = bits.value >> (bits.size - shift_value)
    mask = (1 << bits.size) - 1
    shifted = bits.value << shift_value & mask
    return Bin(shifted | discarded, bits.size)


def xor(bits1: Bin, bits2: Bin):
    return Bin(bits1.value ^ bits2.value, bits1.size)

In [547]:
def generate_subkeys(key: Bin):
    pc1 = permuted_choice_1(key)
    left, right = halve(pc1)

    subkeys: List[Bin] = []
    for i in range(16):
        left = circular_left_shift(left, shift_table[i])
        right = circular_left_shift(right, shift_table[i])
        round_key = permuted_choice_2(left, right)
        subkeys.append(round_key)

    return subkeys


def permuted_choice_1(key: Bin):
    return extract_bits(key, keyp)


def permuted_choice_2(left: Bin, right: Bin):
    bits = fuse_bits(left, right)
    return extract_bits(bits, key_comp)

In [548]:
def feistel_encrypt(plaintext: Bin, key: Bin, test=False,reverse=False):
    subkeys = generate_subkeys(key)
    if reverse:
        subkeys.reverse()
    
    left, right = halve(plaintext)
    round_data: List[List[Bin]] = []
    for i in range(16):
        new_left = right

        sk = subkeys[i]
        magled_right = mangler(right, sk)
        new_right = xor(magled_right, left)

        left = new_left
        right = new_right

        round_data.append([left, right, sk])

    result = fuse_bits(left, right)
    if test:
        return result, round_data

    return result


def mangler(bits: Bin, round_subkey: Bin):
    expanded_right = dbox_expansion(bits)
    xored_right = xor(expanded_right, round_subkey)
    chunks = split_bits(xored_right, 6)
    fused = sbox_substitution(chunks)
    result = pbox_permutation(fused)
    return result


def dbox_expansion(bits: Bin):
    return extract_bits(bits, dbox)


def sbox_substitution(chunks: List[Bin]):
    result = []
    for round, chunk in enumerate(chunks):
        line = extract_bits(chunk, [[1, 6]])
        column = extract_bits(chunk, [[2, 3, 4, 5]])
        sbox_value = sbox[round][line.value][column.value]
        result.append(Bin(sbox_value, 4))

    return fuse_bits(*result)


def pbox_permutation(bits: Bin):
    return extract_bits(bits, pbox)

In [549]:
def des_encryption(plaintext: Bin, key: Bin):
    ciphertext1 = initial_permutation(plaintext)
    ciphertext2 = feistel_encrypt(ciphertext1, key)
    ciphertext3 = swap_halves(ciphertext2)
    ciphertext4 = final_permutation(ciphertext3)
    return ciphertext4


def des_decryption(plaintext: Bin, key: Bin):
    ciphertext1 = initial_permutation(plaintext)
    ciphertext2 = feistel_encrypt(ciphertext1, key, reverse=True)
    ciphertext3 = swap_halves(ciphertext2)
    ciphertext4 = final_permutation(ciphertext3)
    return ciphertext4


def initial_permutation(plaintext: Bin):
    return extract_bits(plaintext, initial_perm)


def final_permutation(ciphertext: Bin):
    return extract_bits(ciphertext, final_perm)

In [550]:
def test_subkey_generation(example_key):
    try:
        expected_keys = [
            Bin(0x194CD072DE8C, 48),
            Bin(0x4568581ABCCE, 48),
            Bin(0x06EDA4ACF5B5, 48),
            Bin(0xDA2D032B6EE3, 48),
            Bin(0x69A629FEC913, 48),
            Bin(0xC1948E87475E, 48),
            Bin(0x708AD2DDB3C0, 48),
            Bin(0x34F822F0C66D, 48),
            Bin(0x84BB4473DCCC, 48),
            Bin(0x02765708B5BF, 48),
            Bin(0x6D5560AF7CA5, 48),
            Bin(0xC2C1E96A4BF3, 48),
            Bin(0x99C31397C91F, 48),
            Bin(0x251B8BC717D0, 48),
            Bin(0x3330C5D9A36D, 48),
            Bin(0x181C5D75C66D, 48),
        ]

        generated_keys = generate_subkeys(example_key)

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return
    
    print("\n\nTESTING SUBKEYS GENERATION".center(40))
    print("-" * 40)
    print("EXPECTED KEY   | GENERATED KEY")
    for g1, g2 in zip(expected_keys, generated_keys):
        print(f"{g1.to_hex()} | {g2.to_hex()}")

    all_match = all(g1 == g2 for g1, g2 in zip(expected_keys, generated_keys))
    if all_match:
        print("\nTest passed: All subkeys match expected values.")
    else:
        print("\nTest failed: Subkeys do not match expected values.")

def test_feistel_encrypt(example_plaintext, example_key):
    try:
        expected_rounds = [
            [Bin(0x18CA18AD, 32), Bin(0x5A78E394, 32), Bin(0x194CD072DE8C, 48)],
            [Bin(0x5A78E394, 32), Bin(0x4A1210F6, 32), Bin(0x4568581ABCCE, 48)],
            [Bin(0x4A1210F6, 32), Bin(0xB8089591, 32), Bin(0x06EDA4ACF5B5, 48)],
            [Bin(0xB8089591, 32), Bin(0x236779C2, 32), Bin(0xDA2D032B6EE3, 48)],
            [Bin(0x236779C2, 32), Bin(0xA15A4B87, 32), Bin(0x69A629FEC913, 48)],
            [Bin(0xA15A4B87, 32), Bin(0x2E8F9C65, 32), Bin(0xC1948E87475E, 48)],
            [Bin(0x2E8F9C65, 32), Bin(0xA9FC20A3, 32), Bin(0x708AD2DDB3C0, 48)],
            [Bin(0xA9FC20A3, 32), Bin(0x308BEE97, 32), Bin(0x34F822F0C66D, 48)],
            [Bin(0x308BEE97, 32), Bin(0x10AF9D37, 32), Bin(0x84BB4473DCCC, 48)],
            [Bin(0x10AF9D37, 32), Bin(0x6CA6CB20, 32), Bin(0x02765708B5BF, 48)],
            [Bin(0x6CA6CB20, 32), Bin(0xFF3C485F, 32), Bin(0x6D5560AF7CA5, 48)],
            [Bin(0xFF3C485F, 32), Bin(0x22A5963B, 32), Bin(0xC2C1E96A4BF3, 48)],
            [Bin(0x22A5963B, 32), Bin(0x387CCDAA, 32), Bin(0x99C31397C91F, 48)],
            [Bin(0x387CCDAA, 32), Bin(0xBD2DD2AB, 32), Bin(0x251B8BC717D0, 48)],
            [Bin(0xBD2DD2AB, 32), Bin(0xCF26B472, 32), Bin(0x3330C5D9A36D, 48)],
            [Bin(0xCF26B472, 32), Bin(0x19BA9212, 32), Bin(0x181C5D75C66D, 48)],
        ]

        initial_permutation = extract_bits(example_plaintext, initial_perm)
        example_key = Bin(0xAABB09182736CCDD, 64)
        rounds_data = feistel_encrypt(initial_permutation, example_key, test=True)[1]

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING FEISTEL ENCRYPTION".center(40))
    print("-" * 40)

    print("EXPECTED ROUND VALUES")
    print("LEFT       | RIGHT      | SUBKEY")
    for l, r, sk in expected_rounds:
        print(f"{l.to_hex()} | {r.to_hex()} | {sk.to_hex()}")

    print("\nGENERATED ROUND VALUES")
    print("LEFT       | RIGHT      | SUBKEY")
    for l, r, sk in rounds_data:
        print(f"{l.to_hex()} | {r.to_hex()} | {sk.to_hex()}")

    print()
    all_match = all(g1 == g2 for g1, g2 in zip(expected_rounds, rounds_data))
    if all_match:
        print("Test passed: All rounds match expected values.")
    else:
        print("Test failed: Rounds do not match expected values.")


def test_des_encryption(example_plaintext, example_key, expected_ciphertext):
    try:
        generated_ciphertext = des_encryption(example_plaintext, example_key)
    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES ENCRYPTION".center(40))
    print("-" * 40)

    print(f"{'Plaintext:':<22}{example_plaintext}")
    print(f"{'Key:':<22}{example_key}")
    print(f"{'Expected Ciphertext:':<22}{expected_ciphertext}")
    print(f"{'Obtained Ciphertext:':<22}{generated_ciphertext}\n")

    assert (
        generated_ciphertext == expected_ciphertext
    ), "Test failed: Ciphertext does not match expected value."

    print("Test passed: Ciphertext matches expected value.")


def test_des_decryption(example_ciphertext, example_key, expected_plaintext):
    try:
        generated_plaintext = des_decryption(example_ciphertext, example_key)

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES DECRYPTION".center(40))
    print("-" * 40)

    print(f"{'Ciphertext:':<22}{example_ciphertext}")
    print(f"{'Key:':<22}{example_key}")
    print(f"{'Expected Plaintext:':<22}{expected_plaintext}")
    print(f"{'Obtained Plaintext:':<22}{generated_plaintext}\n")

    assert (
        generated_plaintext == expected_plaintext
    ), "Test failed: Ciphertext does not match expected value."

    print("Test passed: Ciphertext matches expected value.")

In [552]:
# example_plaintext = Bin(0x123456ABCD132536, 64)
# example_key = Bin(0xAABB09182736CCDD, 64)
# expected_ciphertext = Bin(0xC0B7A8D05F3A829C, 64)

example_plaintext = Bin(0x02468ACEECA86420, 64)
example_key = Bin(0x0F1571C947D9E859, 64)
expected_ciphertext = Bin(0xDA02CE3A89ECAC3B, 64)

test_subkey_generation(example_key)
test_feistel_encrypt(example_plaintext, example_key)
test_des_encryption(example_plaintext, example_key, expected_ciphertext)
test_des_decryption(expected_ciphertext, example_key, example_plaintext)

      

TESTING SUBKEYS GENERATION      
----------------------------------------
EXPECTED KEY   | GENERATED KEY
0x194CD072DE8C | 0x7833C320DA70
0x4568581ABCCE | 0x2B1A74CA48D8
0x06EDA4ACF5B5 | 0x8C78D881D31D
0xDA2D032B6EE3 | 0x1667789316A0
0x69A629FEC913 | 0xCE5D01D80B25
0xC1948E87475E | 0x4BAB4D126A9C
0x708AD2DDB3C0 | 0x09F48B713191
0x34F822F0C66D | 0x710DEAA3202B
0x84BB4473DCCC | 0x129AB83347C3
0x02765708B5BF | 0x9C38661E8103
0x6D5560AF7CA5 | 0xA26E4CC66544
0xC2C1E96A4BF3 | 0x48772468A3C8
0x99C31397C91F | 0xC09D79F0D40B
0x251B8BC717D0 | 0xC5E2634E162A
0x3330C5D9A36D | 0xA3DF829C7968
0x181C5D75C66D | 0xA6120B4D4C25

Test failed: Subkeys do not match expected values.
      

TESTING FEISTEL ENCRYPTION      
----------------------------------------
EXPECTED ROUND VALUES
LEFT       | RIGHT      | SUBKEY
0x18CA18AD | 0x5A78E394 | 0x194CD072DE8C
0x5A78E394 | 0x4A1210F6 | 0x4568581ABCCE
0x4A1210F6 | 0xB8089591 | 0x06EDA4ACF5B5
0xB8089591 | 0x236779C2 | 0xDA2D032B6EE3
0x236779C2 | 0xA15A4B8

Referências

https://www.geeksforgeeks.org/data-encryption-standard-des-set-1/

https://www.geeksforgeeks.org/shift-micro-operations-in-computer-architecture/

https://aprender3.unb.br/pluginfile.php/3076851/mod_resource/content/1/Aula%2008%20-%20Cifra%20de%20Bloco%20-%20DES%20%282%29.pdf